# Dependencies

In [ ]:
pacman::p_load(ggplot2, dplyr)

# Functions

In [ ]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")

In [ ]:
aaFeatures.fx <- function(mixcrfle, aalength, outdir){
    
    aaT <- read.csv(file = "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/aa_attributes.csv", header = T, row.names = 1)
    
    mysample <- gsub(".txt","", mixcrfle)
    mysample <- gsub(".*CLONES_TRB", "", mysample)
    
    f1 <- read.table(file = mixcrfle, sep = "\t", header = T, stringsAsFactors = F)
    
    message(paste("Number of CDR3s in:", mixcrfle))
    print(nrow(f1))
    
    #remove nonproductive
    f1 <- f1[ !grepl("[_]", f1$aaSeqCDR3),]
    f1 <- f1[ !grepl("[*]", f1$aaSeqCDR3),]
    
    message(paste("Number of CDR3s in:", mixcrfle, "nonproductive removed"))
    print(nrow(f1))
    
    f1$aa_length <- nchar(f1$aaSeqCDR3)
    
    f1_subset <- f1[ f1$aa_length == aalength,]
    
    if(nrow(f1_subset) <= 1){
        stop(paste0("less than one CDR3 at length:", aalength, "for file:", mixcrfle))
    }
    
    myaamat_list <- vector("list", aalength)

    myaamat <- matrix(nrow = 20, ncol = 6,
                      dimnames = list(rownames(aaT), 
                                      c("Freq","F1", "F2", "F3", "F4", "F5")))
    for( i in 1:aalength){
    mytab <- table(sapply(strsplit(f1_subset$aaSeqCDR3, ""),"[[",i) )
    myaamat[rownames(mytab),"Freq"] <- mytab
    myaamat[rownames(mytab), 2:6] <- (unlist(aaT[rownames(mytab), 1:5]) * 
                                        myaamat[rownames(mytab),"Freq"])/ sum(myaamat[, "Freq"], na.rm = T) # multiply by number of aminoacid, scale by total cdr3s
    myaamat_list[[i]] <- myaamat
    myaamat[!is.na(myaamat)] <- NA
}
    
    save(myaamat_list, file = paste0(outdir, mysample, "_length", aalength,".RData"))
}

# Paths

In [ ]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [ ]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [ ]:
aaT <- read.csv(file = paste0(datapath, "aa_attributes.csv"), header = T, row.names = 1)

In [ ]:
aaT

In [ ]:
#Table 2 from Atchley et al
#head(aaT)

### Get aaFeatures for each mixcrfile


run locally

In [ ]:
load(file = paste0(datapath,"capTCRseq/TRBfiles.RData"))

In [ ]:
myfilelist <- paste0("~/Desktop/H4H/INTERCEPT/NextSeq/", file_df_dedup_trb$file_list, sep = "")

In [ ]:
head(myfilelist)

In [ ]:
length15 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 15, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

In [ ]:
length14 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 14, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

In [ ]:
length16 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 16, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

In [ ]:
length13 <- lapply(myfilelist, function(x){ 
    tryCatch(
    aaFeatures.fx(x, 13, "~/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/"), error = function(e) e
    )
})

### samples x features for each length

In [ ]:
samples_features_length <- function(inpath, aalength, myfeat){
    mysampleset <- list.files(inpath, pattern = paste(aalength, ".RData", sep = ""), full.names = T)
    
    myLmat <- matrix(ncol = aalength, nrow = length(mysampleset),
                dimnames = list(mysampleset, paste0("L", 1:aalength)))
    
    for(i in 1:nrow(myLmat)){
        load(rownames(myLmat)[i])
        sum_features_aaP <- lapply(myaamat_list, function(x){ colSums(x[,2:6], na.rm = T)})
        names(myaamat_list) <- paste0("L", 1:length(myaamat_list))
        sum_features_aaP_df <- as.data.frame(sum_features_aaP)    
        
        myLmat[i, ] <- unlist(sum_features_aaP_df[myfeat,])
    }
    return(myLmat)
}

In [ ]:
length15_F1 <- samples_features_length( "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/", 15, "F1")

length15_F2 <- samples_features_length( "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/", 15, "F2")
length15_F3 <- samples_features_length( "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/", 15, "F3")
length15_F4 <- samples_features_length( "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/", 15, "F4")
length15_F5 <- samples_features_length( "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures/", 15, "F5")

In [ ]:
rownames(length15_F1) <- gsub("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//", "", rownames(length15_F1))
rownames(length15_F1) <- gsub("_length15.RData", "", rownames(length15_F1))

rownames(length15_F2) <- gsub("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//", "", rownames(length15_F2))
rownames(length15_F2) <- gsub("_length15.RData", "", rownames(length15_F2))

rownames(length15_F3) <- gsub("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//", "", rownames(length15_F3))
rownames(length15_F3) <- gsub("_length15.RData", "", rownames(length15_F3))

rownames(length15_F4) <- gsub("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//", "", rownames(length15_F4))
rownames(length15_F4) <- gsub("_length15.RData", "", rownames(length15_F4))

rownames(length15_F5) <- gsub("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//", "", rownames(length15_F5))
rownames(length15_F5) <- gsub("_length15.RData", "", rownames(length15_F5))

In [ ]:
head(length15_F5)

In [ ]:
save(length15_F1, file = paste0(datapath, "aaFeatures/int_length15_F1.RData"))
save(length15_F2, file = paste0(datapath, "aaFeatures/int_length15_F2.RData"))
save(length15_F3, file = paste0(datapath, "aaFeatures/int_length15_F3.RData"))
save(length15_F4, file = paste0(datapath, "aaFeatures/int_length15_F4.RData"))
save(length15_F5, file = paste0(datapath, "aaFeatures/int_length15_F5.RData"))

### plot

In [ ]:
load(file = paste0(datapath,"capTCRseq/meta_div_TRB.RData"))

In [ ]:
head(meta_div)

In [ ]:
head(length15_F1)

In [ ]:
melt_df <- function(lengthdf, metadta){
    
    lengthdf <- as.data.frame(lengthdf)
    lengthdf$basename <- rownames(lengthdf)
    lengthdf_m <- reshape2::melt(lengthdf)
    
    lengthdf_m$Disease_type <- metadta$Disease_type[ match(lengthdf_m$basename, metadta$basename)]
    lengthdf_m$sampletype <- metadta$sampletype[ match(lengthdf_m$basename, metadta$basename)]
    lengthdf_m$cancergroup <- metadta$cancergroup[ match(lengthdf_m$basename, metadta$basename)]
    lengthdf_m$cycle <- metadta$cycle[ match(lengthdf_m$basename, metadta$basename)]
    return(lengthdf_m)
}

In [ ]:
length15_F1_m <- melt_df(length15_F1, meta_div)
length15_F2_m <- melt_df(length15_F2, meta_div)
length15_F3_m <- melt_df(length15_F3, meta_div)
length15_F4_m <- melt_df(length15_F4, meta_div)
length15_F5_m <- melt_df(length15_F5, meta_div)

In [ ]:
plot.fx <- function(meltedlengthdf, var, colors){
    meltedlengthdf <- meltedlengthdf[!is.na(meltedlengthdf$Disease_type),]
    meltedlengthdf[ order(meltedlengthdf$variable, meltedlengthdf$value),]
    
    cols <- c("variable", var)
    df.summary <- meltedlengthdf %>%
    group_by(across(all_of(cols))) %>% 
    summarise(
        sd = sd(value, na.rm = TRUE),
        meanvalue = mean(value)) %>% ungroup()

  #  print(df.summary)
p1 <- ggplot(aes(x = variable, y = meanvalue, ymin = meanvalue-sd, ymax = meanvalue+sd, color = eval(parse(text = var))), data = df.summary) +
  geom_line(aes(group = eval(parse(text = var)))) +
  geom_errorbar(alpha = 1, width = 0.2, position = position_dodge(0.7)) +
  geom_point(alpha = 1, size = 2, position = position_dodge(0.7)) + myaxis + myplot + 
    theme(legend.position = "bottom", legend.title = element_blank()) +
    scale_color_manual(values = colors) + guides(color=guide_legend(nrow=3,byrow=TRUE))
    return(p1)
}

In [ ]:
#cancergroup
#cycle
#Disease_type
#sampletype

In [ ]:
pdf(paste0(plotpath, "diseasetype_L15.pdf"),
          width = 20, height  = 10)
    plot.fx(length15_F5_m, "Disease_type", cohort_col) + theme(legend.position = "bottom") #+ geom_line()
dev.off()

In [ ]:
plot.fx(length15_F1_m, "cancergroup", group_col) + theme(legend.position = "bottom") #+ geom_line()

remove PBMC

In [ ]:
plot.fx(length15_F5_m[ length15_F5_m$sampletype == "Tumor",], "Disease_type", cohort_col) + 
theme(legend.position = "bottom") 

# PCA

In [ ]:
colnames(length15_F1) <- paste0("F1_", colnames(length15_F1))
colnames(length15_F2) <- paste0("F2_", colnames(length15_F2))
colnames(length15_F3) <- paste0("F3_", colnames(length15_F3))
colnames(length15_F4) <- paste0("F4_", colnames(length15_F4))
colnames(length15_F5) <- paste0("F5_", colnames(length15_F5))

In [ ]:
#bind all
length15 <- cbind(length15_F1, length15_F2, length15_F3, length15_F4, length15_F5)

In [ ]:
#subset to those in metadata
length15 <- length15[ rownames(length15) %in% meta_div$basename,]

In [ ]:
dim(length15)

In [ ]:
rmcol <- colnames(length15)[endsWith(colnames(length15), "_L14" )]

In [ ]:
#length15 <- length15[, !colnames(length15) %in% rmcol]

In [ ]:
colnames(length15)

In [ ]:
#pca
mypca <- prcomp(length15, scale = FALSE)

In [ ]:
library(factoextra)

In [ ]:
#fviz_eig(mypca)

In [ ]:
fviz_pca_ind(mypca,geom = "point",
             col.ind = "cos2", # Color by the quality of representation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
            # repel = TRUE     # Avoid text overlapping
             )

In [ ]:
fviz_pca_var(mypca,
             col.var = "contrib", # Color by contributions to the PC
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE     # Avoid text overlapping
             )

In [ ]:
library(tidyverse)
library(palmerpenguins)
library(Rtsne)

In [ ]:
#length15 <- length15[, !grepl("L3", colnames(length15))]

In [ ]:
colnames(length15)

In [ ]:
set.seed(142)
tSNE_fit <- length15 %>%
  #scale() %>% 
  Rtsne(perplexity = 15)

In [ ]:
tSNE_df <- tSNE_fit$Y

rownames(tSNE_df) <- rownames(length15)

In [ ]:
colnames(tSNE_df) <- c("tSNE1", "tSNE2")

In [ ]:
tSNE_df <- as.data.frame(tSNE_df)

In [ ]:
head(tSNE_df)

In [ ]:
meta_div_t <- meta_div

In [ ]:
meta_div_t$tSNE1 <- tSNE_df$tSNE1[ match(meta_div_t$basename, rownames(tSNE_df))]
meta_div_t$tSNE2 <- tSNE_df$tSNE2[ match(meta_div_t$basename, rownames(tSNE_df))]

In [ ]:
meta_div_t <- meta_div_t[!is.na(meta_div_t$tSNE1),]

In [ ]:
colnames(meta_div_t)

In [ ]:
ggplot(data = meta_div_t[ !is.na(meta_div_t$TE.),], aes(x = tSNE1, y = tSNE2, color = Naïve.) ) +
  geom_point() +
  myplot+myaxis + theme(legend.position = "bottom", legend.title = element_blank()) + 
guides(color = guide_legend(nrow = 2))# + scale_color_manual(values = cycle_col)

In [ ]:
head(length15)

In [ ]:
tsne(t(length15), perplex = 5)

In [ ]:
load("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//CHP_11YO-M_PBMC_DNA_length15.RData")

In [ ]:
load("/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/aaFeatures//CHP_10YO-M_PBMC_DNA_length15.RData")

In [ ]:
myaamat_list

In [ ]:
myaamat_list

In [ ]:
load(paste0(h4hpath, "aaFeatures/CHP_358-02-PBMC-DNA_length15.RData"))

In [ ]:
sum_features_aaP <- lapply(myaamat_list, function(x){ colSums(x[,2:6], na.rm = T)})
names(myaamat_list) <- paste0("L", 1:length(myaamat_list))
sum_features_aaP_df <- as.data.frame(sum_features_aaP)

In [ ]:
sum_features_aaP_df

In [ ]:
aalength <- 15
myfeat <- "F1"

In [ ]:
mysampleset <- list.files(paste0(h4hpath, "aaFeatures/"), pattern = paste(aalength, ".RData", sep = ""), full.names = T)

In [ ]:
myLmat <- matrix(ncol = aalength, nrow = length(mysampleset),
                dimnames = list(mysampleset, paste0("L", 1:aalength)))

In [ ]:
myLmat[1, ] <- unlist(sum_features_aaP_df[myfeat,])

In [ ]:
head(myLmat)

# NMI

In [ ]:
mat <- sapply(as.character(c("CASSFYGREADTQYF", "CASSVVGVYNSPLHF", "CASSLTRPSGNTIYF",
                            "CASSYQAGVGETQYF")), function(x){
      x <- as.character(x);
      x <- strsplit(x, "");
      x <- unlist( x )
   })

mat <- t(mat)

In [ ]:
 res <- data.frame()
   for( i in 1:15 ){
   for( k in 1:15 ){
      print(paste0( i, ":", k))
      d1 <- mat[, c(i,k)]
      d1 <- as.data.frame(d1)
      colnames(d1) <- c("X","Y")
      d1 <- d1[ d1$X!="NA" &  d1$Y!="NA",] #revemo NA
      
      xlist <- as.character( unique(d1$X) )
      ylist <- as.character( unique(d1$Y) )
      
      #make frequency table
      freq <- matrix(0, nrow=length(xlist), ncol=length(ylist))
      row.names(freq) <- xlist
      colnames(freq) <- ylist
      
      for( x in xlist ){
      for( y in ylist ){
         both <- subset(d1, X==x & Y==y)
         freq[ x, y ] <- nrow( both )
      }
      }
      
      MI <- mi.empirical(freq, unit=c("log"))
      Hx <- entropy.empirical( colSums(freq) )
      Hy <- entropy.empirical( rowSums(freq) )
      NMI <- 2 * MI / ( Hx + Hy ) #normalized mutual entropy
      
      dump <- data.frame(i, k, MI, Hx, Hy, NMI)
      res <- rbind(res, dump)
      
   }
   }

In [ ]:
res